In [70]:
import hydra
from tqdm import tqdm
from omegaconf import DictConfig, OmegaConf
import tensorflow as tf
from dataset import Dataset
from model import UCCModel
from logging import getLogger
import os
import sys
from train import grad, eval
import re

In [71]:
output_path = "outputs/2024-03-28/10-18-51/"
model_name = "model_weights.h5"
cfg = OmegaConf.load(os.path.join(output_path, ".hydra\\config.yaml"))
args = cfg.args
dataset = Dataset(
        num_instances=args.num_instances,
        num_samples_per_class=args.num_samples_per_class,
        digit_arr= [x for x in range(10)],
        ucc_start=args.ucc_start,
        ucc_end=args.ucc_end
        )
optimizer = tf.keras.optimizers.Adam(learning_rate=args.learning_rate)
model = UCCModel(cfg)
model.build(input_shape=())
model.load_weights(filepath=os.path.join(output_path, model_name))

x_train shape: (50000, 28, 28, 1)
50000 train samples
10000 val samples
digit0:0, num_train:4936, num_val:987
digit1:1, num_train:5619, num_val:1123
digit2:2, num_train:4965, num_val:993
digit3:3, num_train:5110, num_val:1021
digit4:4, num_train:4869, num_val:973
digit5:5, num_train:4514, num_val:907
digit6:6, num_train:4932, num_val:986
digit7:7, num_train:5221, num_val:1044
digit8:8, num_train:4876, num_val:975
digit9:9, num_train:4958, num_val:991


In [75]:
def train(model, optimizer, dataset, args, save_path):
    train_summary_writer = tf.summary.create_file_writer(os.path.join(save_path, "train_logs"))
    val_summary_writer = tf.summary.create_file_writer(os.path.join(save_path, "val_logs"))
    with open(os.path.join(output_path, "train.log")) as file:
        last_line = file.readlines()[-1]
    a = re.search(r"Step.*,", last_line)
    current_step = int(last_line[a.start()+5: a.end()-1])
    logger = getLogger()
    num_steps = args.train_num_steps
    physical_devices = tf.config.list_physical_devices('GPU')
    best_eval_acc = 0
    if physical_devices:
        with tf.device('/device:GPU:0'):
            for i in tqdm(range(current_step, 80000)):

                _, [labels, inputs] = dataset.next_batch_train()
                loss_dict, grads = grad(model, inputs, labels)
                with train_summary_writer.as_default():
                    for key in loss_dict.keys():      
                        tf.summary.scalar(key, loss_dict[key], step=i)
                    train_summary_writer.flush()
                    
                optimizer.apply_gradients(zip(grads, model.trainable_variables))
                if i%10 == 0:
                    eval_loss_dict = eval(model, dataset)
                    with val_summary_writer.as_default():
                        for key in eval_loss_dict.keys():      
                            tf.summary.scalar(key, loss_dict[key], step=i)
                        val_summary_writer.flush()
                    text = (f"Step {i+1}, training: ucc_acc={loss_dict['ucc_acc']} weighted_loss={loss_dict['weighted_loss']} ucc_loss={loss_dict['ucc_loss']} ae_loss={loss_dict['ae_loss']} eval: ucc_acc={eval_loss_dict['ucc_acc']} weighted_loss={eval_loss_dict['weighted_loss']} ucc_loss={eval_loss_dict['ucc_loss']} ae_loss={eval_loss_dict['ae_loss']}")
                    print(text)
                    with open(os.path.join(output_path, "train.log"), "a") as file:
                        file.write(text+"\n")
                    acc = eval_loss_dict['ucc_acc']
                    if acc>= best_eval_acc:
                        best_eval_acc = acc
                        model.save_weights(os.path.join(save_path,"model_weights.h5"), save_format="h5")

In [76]:
train(model=model, save_path=output_path, dataset=dataset, args=args, optimizer=optimizer)

  0%|          | 0/20009 [00:00<?, ?it/s]

  0%|          | 10/20009 [00:03<2:02:55,  2.71it/s]

Step 60001, training: ucc_acc=1.0 weighted_loss=0.041770149022340775 ucc_loss=0.041770149022340775 ae_loss=0 eval: ucc_acc=1.0 weighted_loss=0.020018266513943672 ucc_loss=0.020018266513943672 ae_loss=0


  0%|          | 20/20009 [00:06<2:00:39,  2.76it/s]

Step 60011, training: ucc_acc=1.0 weighted_loss=0.000492640130687505 ucc_loss=0.000492640130687505 ae_loss=0 eval: ucc_acc=1.0 weighted_loss=0.0007181764813140035 ucc_loss=0.0007181764813140035 ae_loss=0


...

100%|█████████▉| 19990/20009 [1:49:23<00:07,  2.52it/s]

Step 79981, training: ucc_acc=1.0 weighted_loss=0.005253890994936228 ucc_loss=0.005253890994936228 ae_loss=0 eval: ucc_acc=1.0 weighted_loss=0.0294803436845541 ucc_loss=0.0294803436845541 ae_loss=0


100%|█████████▉| 20000/20009 [1:49:26<00:03,  2.85it/s]

Step 79991, training: ucc_acc=1.0 weighted_loss=0.0024361349642276764 ucc_loss=0.0024361349642276764 ae_loss=0 eval: ucc_acc=1.0 weighted_loss=0.015408285893499851 ucc_loss=0.015408285893499851 ae_loss=0


100%|██████████| 20009/20009 [1:49:29<00:00,  3.05it/s]
